# Passo a passo de Transformadores

A arquitetura usada foi a implementada originalmente no repositório [_Vanilla Transformer_](https://github.com/arxyzan/vanilla-transformer) com autoria de [Aryan Shekarlaban](https://github.com/arxyzan). A arquitetura, como descrito pelo autor, almeja fazer uma implementação simples e fiel à arquitetura proposta pelo artigo original [_Attention is all you need_](../../papers/attention_is_all_you_need.pdf).

<center>
<img src='assets/transformers_architecture.png' height=400/>
</center>

O código possui 2 modelos, um transformador completo na classe `Transformer`,e um classificador utilizando apenas o _encoder_ na classe `EncoderClassifier`. Além disso possui 2 "blocos", um _encoder_ na classe `Encoder`, um _decoder_ na classe `Decoder`. Finalmente possui 3 tipos de camada, a camada de _auto-atenção_ na classe `MultiHeadAttention`, a camada de _encoder_ na classe `EncoderLayer`, e a camada de _decoder_ na classe `Decoder`. Abaixo as classes são descritas com maior detalhe.

Esse notebook descreve passo a passo o código usado para implementar transformadores com PyTorch.


In [1]:
# Importando pacotes
import torch
import torch.nn as nn

## Camadas


### `MultiHeadAttention`


A implementação da classe segue a seguinte arquitetura:

<center>
<img src='assets/multihead_attention.png' height=300/>
</center>

Essa classe implementa uma camada de _Multi-Head Self Attention_, recebe como entrada 3 parâmetros:

- `embed_dim`: Representa a dimensão total do embedding de cada token, que nada mais é do que o tamanho do vetor de características para cada token após a camada de embedding.

- `n_heads`: Representa o número de cabeças de atenção. Cada _head_ é um mecanismo de atenção independente que processa diferentes subespaços de uma mesma entrada. Isso acontece devido a diferentes pesos em cada _head_.

- `dropout`: probabilidade _p_ de zerar um elemento do vetor de entrada.

O algoritmo de atenção é descrito pela imagem abaixo:

<center>
<img src='assets/attention.png' height=300/>
</center>

#### Projeção linear

O primeiro passo no método `forward` é fazer projeções lineares dos vetores de _queries, keys_ e _values_, de modo a permitir que o modelo aprenda diferentes representações para cada um dos papeis em uma mesma entrada. Essa etapa não altera a dimensão dos vetores.

```python
        # [batch_size, seq_len, embed_dim] -> [batch_size, seq_len, embed_dim]
        Q = self.queries(q)
        K = self.keys(k)
        V = self.values(v)
```

#### Redimensionamento e separação de _heads_

Em seguida é feita a preparação para o algoritmo de atenção, com o redimensionamento dos tensores `Q`, `K` e `V` para separação das _heads_. Os tensores são redimensionados, em um primeiro momento, de $(batch\ size \times seq\ len \times embed\ dim)$ para $(batch\ size \times seq\ len \times embed\ dim \times head\ dim)$. Esse redimensionamento é feito pelo método `.view`.

Após o redimensionamento, é feita uma permutação das dimensões de modo que o número de cabeças seja a segunda dimensão. Isso mais uma vez muda as dimensões do vetor, dessa vez de $(batch\ size \times head\ dim \times seq\ len \times head\ dim)$. A permutação é feita pelo método `.permute`

```python
        # [batch_size, seq_len, embed_dim] -> [batch_size, head_dim, seq_len, head_dim]
        Q = Q.view(N, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(N, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(N, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)

```

#### Queries X Keys e normalização

Feito o redimensionamento, o próximo passo é fazer a multiplicação das _queries_ com as _keys_. Para multiplicar esses dois tensores, é necessário primeiro fazer um redimensionamento do tensor de _keys_, botando a dimensão $head\ dim$ no eixo $2$ ao invés do eixo $3$.

Isso é feito de modo a fazer produto escalar do vetor de embedding de cada token (para cada cabeça na dimensão $head\ dim$ das _queries_), com o vetor de embedding (também para cada cabeça na dimensão $head\ dim$ das _keys_) de todos os tokens da sequência. Ou seja, para cada token da _query_, calculamos a similaridade com cada token da _key_ ao longo de uma sequência.

Essa multiplicação, chamada _energia_ (_energy_), é então normalizada pela raiz quadrada da $embed\ dim$, ajudando a estabilizar o gradiente ao longo do treinamento.

```python
        # [batch_size, head_dim, seq_len, head_dim] -> [batch_size, n_heads, seq_len, seq_len]]
        energy = (Q.matmul(K.permute(0, 1, 3, 2))) / self.scale
```

#### Aplicação de máscara e softmax

A máscara controla quais posições da sequência o modelo pode ou não considerar durante o cálculo da atenção. Nesse caso a ideia é remover a influência de tokens de padding adicionados à sequência. Isso é feito com o método `.masked_fill`, substituindo valores de $0$ por valores extremamente pequenos $-1\times10^{20}$, de modo que após a passagem pela função _softmax_ a probabilidade correspondente seja praticamente zero.

A função _softmax_ é aplicada ao longo da última dimensão do tensor, de modo a transformar os scores de atenção (`energy`) em probabilidades.

```python
        # [batch_size, n_heads, seq_len, seq_len]
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e20)

        attention = energy.softmax(-1)
```

#### Attention X Values

A multiplicação entre os valores de _atenção_ (_attention_) e _values_ mensura quanto os tokens devem prestar atenção uns aos outros. Essa multiplicação é feita juntamente a um dropout nos vetores de atenção.

```python
        # [batch_size, n_heads, seq_len, seq_len]
        x = self.dropout(attention).matmul(V)
```

#### Redimensionamento e concatenação

Novamente os tensores passam por um redimensionamento, devolvendo a dimensão $head\ dim$ ao final do tensor. O método `.contiguous` garante que o tensor seja armazenado de forma contínua, para as operações subsequentes. Após o redimensionamento, as _heads_ são concatenadas, achatando a segunda e última dimensão, com todas as representações produzidas por cada uma das _heads_ colocadas lado a lado, formando um único vetor de dimensão $embed\ dim$.

Após a concatenação, o tensor passa por uma outra camada linear, de modo a "misturar" as informações das diferentes heads e projetar o resultado de volta ao espaço de embedding original.

```python
        # [batch_size, n_heads, seq_len, seq_len] -> [batch_size, seq_len, n_heads, head_dim]
        x = x.permute(0, 2, 1, 3).contiguous()
        x = x.view(N, -1, self.embed_dim)
        x = self.proj(x)
```


In [2]:
class MultiHeadAttention(nn.Module):
    '''
    Self-attention layer for Transformer.
    '''

    def __init__(
        self,
        embed_dim: int,
        n_heads: int,
        dropout: float,
    ) -> None:
        '''
        Class initializer.

        Args:
            vocab_size: Size of vocabulary.
            n_layers: Number of Encoder layers.
            n_classes: Number of classes for output.
            ff_hid_dim: Size of hidden dimension in feed-forward layer.
            embed_dim: Size of embedding dimension.
            n_heads: Number of self-attention heads.
            max_length: Maximum length of vector.
            pad_idx: Index of padding token.
            dropout: p of dropout.
            device: Computing device
        '''
        super().__init__()
        self.head_dim = embed_dim // n_heads
        self.n_heads = n_heads
        self.embed_dim = embed_dim
        self.scale = embed_dim**0.5

        self.keys = nn.Linear(embed_dim, embed_dim)
        self.queries = nn.Linear(embed_dim, embed_dim)
        self.values = nn.Linear(embed_dim, embed_dim)
        self.proj = nn.Linear(embed_dim, embed_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(
        self,
        q: torch.Tensor,
        k: torch.Tensor,
        v: torch.Tensor,
        mask: torch.Tensor | None = None,
    ) -> torch.Tensor:
        '''
        Forward pass through architecture.

        Args:
            q: Tensor of queries.
            k: Tensor of keys.
            v: Tensor of values.
            mask: Energy mask.
        '''
        N = q.shape[0]

        Q = self.queries(q)
        K = self.keys(k)
        V = self.values(v)

        Q = Q.view(N, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(N, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(N, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)

        energy = (Q.matmul(K.permute(0, 1, 3, 2))) / self.scale
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e20)

        attention = energy.softmax(-1)

        x = self.dropout(attention).matmul(V)
        x = x.permute(0, 2, 1, 3).contiguous()
        x = x.view(N, -1, self.embed_dim)
        x = self.proj(x)

        return x

### `EncoderLayer`


A classe `EncoderLayer` segue a arquitetura abaixo:

<center>
<img src='assets/encoder_layer.png' height=300/>
</center>

A camada recebe 4 parâmetros como entrada:

- `embed_dim`: Representa a dimensão total do embedding de cada token, que nada mais é do que o tamanho do vetor de características para cada token após a camada de embedding.

- `n_heads`: Representa o número de cabeças de atenção. Cada _head_ é um mecanismo de atenção independente que processa diferentes subespaços de uma mesma entrada. Isso acontece devido a diferentes pesos em cada _head_.

- `ff_hid_dim`: Representa o número de unidades da camada oculta dentro do bloco feed-forward do _encoder_.

- `dropout`: Probabilidade _p_ de zerar um elemento do vetor de entrada.


In [3]:
class EncoderLayer(nn.Module):
    '''
    Encoder layer with self-attention.
    '''

    def __init__(
        self, embed_dim: int, n_heads: int, ff_hid_dim: int, dropout: float
    ) -> None:
        '''
        Class initializer.

        Args:
            embed_dim: Size of embedding dimension.
            n_heads: Number of self-attention heads.
            ff_hid_dim: Size of hidden dimension in feed-forward layer.
            device: Computing device
        '''
        super().__init__()
        self.attention = MultiHeadAttention(
            embed_dim=embed_dim, n_heads=n_heads, dropout=dropout
        )

        self.norm_1 = nn.LayerNorm(embed_dim)
        self.norm_2 = nn.LayerNorm(embed_dim)

        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, ff_hid_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(ff_hid_dim, embed_dim),
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, src: torch.Tensor, mask: torch.Tensor) -> torch.Tensor:
        '''
        Forward pass through architecture.

        Args:
            src: Source tensor.
            mask: Source tensor mask.
        '''
        attention = self.attention(src, src, src, mask)
        x = self.norm_1(attention + self.dropout(src))

        out = self.mlp(x)
        out = self.norm_2(out + self.dropout(x))

        return out

### `DecoderLayer`


A classe `DecoderLayer` segue a arquitetura abaixo:

<center>
<img src='assets/decoder_layer.png' height=300/>
</center>

A camada recebe 4 parâmetros como entrada:

- `embed_dim`: Representa a dimensão total do embedding de cada token, que nada mais é do que o tamanho do vetor de características para cada token após a camada de embedding.

- `n_heads`: Representa o número de cabeças de atenção. Cada _head_ é um mecanismo de atenção independente que processa diferentes subespaços de uma mesma entrada. Isso acontece devido a diferentes pesos em cada _head_.

- `ff_hid_dim`: Representa o número de unidades da camada oculta dentro do bloco feed-forward do _decoder_.

- `dropout`: Probabilidade _p_ de zerar um elemento do vetor de entrada.


In [4]:
class DecoderLayer(nn.Module):
    '''
    Decoder layer with self-attention.
    '''

    def __init__(
        self,
        embed_dim: int,
        n_heads: int,
        ff_hid_dim: int,
        dropout: float,
    ) -> None:
        '''
        Class initializer.

        Args:
            embed_dim: Size of embedding dimension.
            n_heads: Number of self-attention heads.
            ff_hid_dim: Size of hidden dimension in feed-forward layer.
            dropout: p of dropout.
        '''

        super().__init__()
        self.attention = MultiHeadAttention(
            embed_dim=embed_dim, n_heads=n_heads, dropout=dropout
        )
        self.joint_attention = MultiHeadAttention(
            embed_dim=embed_dim, n_heads=n_heads, dropout=dropout
        )

        self.norm_1 = nn.LayerNorm(embed_dim)
        self.norm_2 = nn.LayerNorm(embed_dim)
        self.norm_3 = nn.LayerNorm(embed_dim)

        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, ff_hid_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(ff_hid_dim, embed_dim),
        )

        self.dropout = nn.Dropout(dropout)

    def forward(
        self,
        trg: torch.Tensor,
        src: torch.Tensor,
        trg_mask: torch.Tensor,
        src_mask: torch.Tensor,
    ) -> torch.Tensor:
        '''
        Forward pass through architecture.

        Args:
            trg: Target tensor.
            src: Source tensor.
            trg_mask: Target tensor mask.
            src_mask: Source tensor mask.
        '''
        trg_attention = self.attention(trg, trg, trg, trg_mask)
        trg = self.norm_1(trg + self.dropout(trg_attention))

        joint_attention = self.attention(trg, src, src, src_mask)
        trg = self.norm_2(trg + self.dropout(joint_attention))

        out = self.mlp(trg)
        out = self.norm_2(trg + self.dropout(out))

        return out

## Blocks


### `Encoder`


A classe `Encoder` segue a arquitetura abaixo:

<center>
<img src='assets/encoder.png' height=300/>
</center>

A camada recebe 8 parâmetros como entrada:

- `vocab_size`: Tamanho do vocabulário de entrada.

- `embed_dim`: Representa a dimensão total do embedding de cada token, que nada mais é do que o tamanho do vetor de características para cada token após a camada de embedding.

- `n_layers`: Número de camadas de `EncoderLayer` empilhadas no modelo. Cada camada possibilita o modelo aprender com maior profundidade a capacidade de modelagem.

- `n_heads`: Representa o número de cabeças de atenção. Cada _head_ é um mecanismo de atenção independente que processa diferentes subespaços de uma mesma entrada. Isso acontece devido a diferentes pesos em cada _head_.

- `ff_hid_dim`: Representa o número de unidades da camada oculta dentro do bloco feed-forward do _encoder_.

- `max_length`: Tamanho máximo da sequência de entrada suportada pelo modelo. É usada para criar embeddings posicionais.

- `dropout`: Probabilidade _p_ de zerar um elemento do vetor de entrada.

- `device`: Dispositivo de computação onde os tensores serão alocados.


In [5]:
class Encoder(nn.Module):
    '''
    Encoder Block with self-attention.
    '''

    def __init__(
        self,
        vocab_size: int,
        embed_dim: int,
        n_layers: int,
        n_heads: int,
        ff_hid_dim: int,
        max_length: int,
        dropout: float,
        device: str,
    ) -> None:
        '''
        Class initializer.

        Args:
            vocab_size: Size of vocabulary.
            embed_dim: Size of embedding dimension.
            n_layers: Number of Encoder layers.
            n_heads: Number of self-attention heads.
            ff_hid_dim: Size of hidden dimension in feed-forward layer.
            max_length: Maximum length of vector.
            dropout: p of dropout.
            device: Computing device
        '''
        super().__init__()
        self.device = device
        self.scale = embed_dim**0.5

        self.tok_emb = nn.Embedding(vocab_size, embed_dim)
        self.pos_emb = nn.Embedding(max_length, embed_dim)

        self.blocks = nn.ModuleList(
            [
                EncoderLayer(
                    embed_dim=embed_dim,
                    n_heads=n_heads,
                    ff_hid_dim=ff_hid_dim,
                    dropout=dropout,
                )
                for _ in range(n_layers)
            ]
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, src: torch.Tensor, mask: torch.Tensor) -> torch.Tensor:
        '''
        Forward pass through architecture.

        Args:
            src: Source tensor.
            mask: Source tensor mask.
        '''

        N, seq_len = src.shape

        positions = torch.arange(0, seq_len).expand(N, seq_len).to(self.device)
        pos_embeddings = self.pos_emb(positions)
        tok_embeddings = self.tok_emb(src) * self.scale
        out = self.dropout(tok_embeddings + pos_embeddings)

        for block in self.blocks:
            out = block(out, mask)

        return out

### `Decoder`


A classe `Decoder` segue a arquitetura abaixo:

<center>
<img src='assets/decoder.png' height=300/>
</center>

A camada recebe 8 parâmetros como entrada:

- `vocab_size`: Tamanho do vocabulário de entrada.

- `embed_dim`: Representa a dimensão total do embedding de cada token, que nada mais é do que o tamanho do vetor de características para cada token após a camada de embedding.

- `n_layers`: Número de camadas de `DecoderLayer` empilhadas no modelo. Cada camada possibilita o modelo aprender com maior profundidade a capacidade de modelagem.

- `n_heads`: Representa o número de cabeças de atenção. Cada _head_ é um mecanismo de atenção independente que processa diferentes subespaços de uma mesma entrada. Isso acontece devido a diferentes pesos em cada _head_.

- `ff_hid_dim`: Representa o número de unidades da camada oculta dentro do bloco feed-forward do _decoder_.

- `max_length`: Tamanho máximo da sequência de entrada suportada pelo modelo. É usada para criar embeddings posicionais.

- `dropout`: Probabilidade _p_ de zerar um elemento do vetor de entrada.

- `device`: Dispositivo de computação onde os tensores serão alocados.

A segunda camada de atenção recebe como _queries_ e _keys_, a saída da camada de encoder.


In [6]:
class Decoder(nn.Module):
    '''
    Decoder block with self-attention
    '''

    def __init__(
        self,
        vocab_size: int,
        embed_dim: int,
        n_layers: int,
        n_heads: int,
        ff_hid_dim: int,
        max_length: int,
        dropout: float,
        device: str,
    ) -> None:
        '''
        Class initializer.

        Args:
            vocab_size: Size of vocabulary.
            embed_dim: Size of embedding dimension.
            n_layers: Number of Encoder layers.
            n_heads: Number of self-attention heads.
            ff_hid_dim: Size of hidden dimension in feed-forward layer.
            max_length: Maximum length of vector.
            dropout: p of dropout.
            device: Computing device
        '''
        super().__init__()
        self.device = device
        self.scale = embed_dim**0.5

        self.tok_emb = nn.Embedding(vocab_size, embed_dim)
        self.pos_emb = nn.Embedding(max_length, embed_dim)

        self.blocks = nn.ModuleList(
            [
                DecoderLayer(
                    embed_dim=embed_dim,
                    n_heads=n_heads,
                    ff_hid_dim=ff_hid_dim,
                    dropout=dropout,
                )
                for _ in range(n_layers)
            ]
        )

        self.fc = nn.Linear(embed_dim, vocab_size)

        self.dropout = nn.Dropout(dropout)

    def forward(
        self,
        trg: torch.Tensor,
        src: torch.Tensor,
        trg_mask: torch.Tensor,
        src_mask: torch.Tensor,
    ) -> torch.Tensor:
        '''
        Forward pass through architecture.

        Args:
            trg: Target tensor.
            src: Source tensor.
            trg_mask: Target tensor mask.
            src_mask: Source tensor mask.
        '''
        N, trg_len = trg.shape

        positions = torch.arange(0, trg_len).expand(N, trg_len).to(self.device)
        pos_embeddings = self.pos_emb(positions)
        tok_embeddings = self.tok_emb(trg) * self.scale
        trg = self.dropout(tok_embeddings + pos_embeddings)

        for block in self.blocks:
            trg = block(trg, src, trg_mask, src_mask)

        out = self.fc(trg)
        return out

## Modelos


### `Transformer`


A classe `Transformer` segue a arquitetura abaixo:

<center>
<img src='assets/transformers_architecture.png' height=300/>
</center>

A camada recebe 11 parâmetros como entrada:

- `src_vocab_size`: Tamanho do vocabulário da fonte.

- `trg_vocab_size`: Tamanho do vocabulário do alvo.

- `src_pad_idx`: Índice do token de padding da fonte.

- `trg_pad_idx`: Índice do token de padding do alvo.

- `embed_dim`: Representa a dimensão total do embedding de cada token, que nada mais é do que o tamanho do vetor de características para cada token após a camada de embedding.

- `n_layers`: Número de camadas de `DecoderLayer` empilhadas no modelo. Cada camada possibilita o modelo aprender com maior profundidade a capacidade de modelagem.

- `n_heads`: Representa o número de cabeças de atenção. Cada _head_ é um mecanismo de atenção independente que processa diferentes subespaços de uma mesma entrada. Isso acontece devido a diferentes pesos em cada _head_.

- `ff_hid_dim`: Representa o número de unidades da camada oculta dentro do bloco feed-forward do _Decoder_.

- `max_length`: Tamanho máximo da sequência de entrada suportada pelo modelo. É usada para criar embeddings posicionais.

- `dropout`: Probabilidade _p_ de zerar um elemento do vetor de entrada.

- `device`: Dispositivo de computação onde os tensores serão alocados.

A classe também possui 2 métodos além do `forward`:

- `src_mask`: Cria uma máscara de padding de um tensor fonte.

- `trg_mask`: Cria uma look-ahead mask no tensor alvo.


In [7]:
class Transformer(nn.Module):
    '''
    Transformer architecture with Encoder and Decoder blocks.
    '''

    def __init__(
        self,
        src_vocab_size: int,
        trg_vocab_size: int,
        src_pad_idx: int,
        trg_pad_idx: int,
        embed_dim: int,
        n_layers: int,
        n_heads: int,
        ff_hid_dim: int,
        max_length: int,
        dropout: float,
        device: str,
    ) -> None:
        '''
        Class initializer.

        Args:
            src_vocab_size: Size of source vocabulary.
            trg_vocab_size: Size of target vocabulary.
            src_pad_idx: Index of source padding token.
            trg_pad_idx: Index of target padding token.
            embed_dim: Size of embedding dimension.
            n_layers: Number of Encoder layers.
            n_heads: Number of self-attention heads.
            ff_hid_dim: Size of hidden dimension in feed-forward layer.
            max_length: Maximum length of vector.
            dropout: p of dropout.
            device: Computing device
        '''
        super().__init__()

        self.encoder = Encoder(
            vocab_size=src_vocab_size,
            embed_dim=embed_dim,
            n_layers=n_layers,
            n_heads=n_heads,
            ff_hid_dim=ff_hid_dim,
            max_length=max_length,
            dropout=dropout,
            device=device,
        )
        self.decoder = Decoder(
            vocab_size=trg_vocab_size,
            embed_dim=embed_dim,
            n_layers=n_layers,
            n_heads=n_heads,
            ff_hid_dim=ff_hid_dim,
            max_length=max_length,
            dropout=dropout,
            device=device,
        )

        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx

        self.device = device

    def src_mask(self, src: torch.Tensor) -> torch.Tensor:
        '''
        Creates mask for source tensor.
        '''
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        return src_mask.to(self.device)

    def trg_mask(self, trg: torch.Tensor) -> torch.Tensor:
        '''
        Creates mask for target tensor.
        '''
        N, trg_len = trg.shape

        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)
        trg_pad_mask = (
            torch.tril(torch.ones(trg_len, trg_len)).bool().to(self.device)
            & trg_pad_mask
        )

        return trg_pad_mask

    def forward(self, src: torch.Tensor, trg: torch.Tensor) -> torch.Tensor:
        '''
        Forward pass through architecture.

        Args:
            trg: Target tensor.
            src: Source tensor.
        '''
        src_mask = self.src_mask(src)
        trg_mask = self.trg_mask(trg)

        encoded = self.encoder(src, src_mask)
        decoded = self.decoder(trg, encoded, trg_mask, src_mask)

        return decoded

### `EncoderClassifier`


A classe recebe 10 parâmetros como entrada:

- `vocab_size`: Tamanho do vocabulário da entrada.

- `n_layers`: Número de camadas de `DecoderLayer` empilhadas no modelo. Cada camada possibilita o modelo aprender com maior profundidade a capacidade de modelagem.

- `n_classes`: Número de classes a prever.

- `embed_dim`: Representa a dimensão total do embedding de cada token, que nada mais é do que o tamanho do vetor de características para cada token após a camada de embedding.

- `n_heads`: Representa o número de cabeças de atenção. Cada _head_ é um mecanismo de atenção independente que processa diferentes subespaços de uma mesma entrada. Isso acontece devido a diferentes pesos em cada _head_.

- `ff_hid_dim`: Representa o número de unidades da camada oculta dentro do bloco feed-forward do _Decoder_.

- `max_length`: Tamanho máximo da sequência de entrada suportada pelo modelo. É usada para criar embeddings posicionais.

- `pad_idx`: Índice do token de padding da entrada.

- `dropout`: Probabilidade _p_ de zerar um elemento do vetor de entrada.

- `device`: Dispositivo de computação onde os tensores serão alocados.

A classe também possui 1 método além do `forward`:

- `mask`: Cria uma máscara de padding do tensor de entrada.

Esse classificador se utiliza apenas da classe `Encoder`, adicionando ao fim uma camada completamente conectada para previsão das diferentes possíveis classes. O método forward cria a máscara da entrada, passa ela pelo encoder, e em seguida por uma camada de _dropout_.

Após isso, é aplicado um _max-pooling_ à saída do _encoder_, redimensionando o tensor de $(batch\ size \times seq\ len \times embed\ dim)$ para $(batch\ size  \times embed\ dim)$. Finalmente a saída do _max-pooling_ passa pela camada completamente conectada, retornando os _logits_ (probabilidades) de cada classe.


In [8]:
class EncoderClassifier(nn.Module):
    '''
    Custom classifier with a Transformer's Encoder block.
    '''

    def __init__(
        self,
        vocab_size: int,
        n_layers: int,
        n_classes: int,
        embed_dim: int,
        n_heads: int,
        ff_hid_dim: int,
        max_length: int,
        pad_idx: int,
        dropout: float,
        device: str,
    ) -> None:
        '''
        Class initializer.

        Args:
            vocab_size: Size of vocabulary.
            n_layers: Number of Encoder layers.
            n_classes: Number of classes for output.
            embed_dim: Size of embedding dimension.
            n_heads: Number of self-attention heads.
            ff_hid_dim: Size of hidden dimension in feed-forward layer of Encoder.
            max_length: Maximum length of vector.
            pad_idx: Index of padding token.
            dropout: p of dropout.
            device: Computing device
        '''
        super().__init__()
        self.pad_idx = pad_idx

        self.encoder = Encoder(
            vocab_size=vocab_size,
            embed_dim=embed_dim,
            n_layers=n_layers,
            n_heads=n_heads,
            ff_hid_dim=ff_hid_dim,
            max_length=max_length,
            dropout=dropout,
            device=device,
        )

        self.linear = nn.Linear(
            in_features=embed_dim,
            out_features=n_classes,
        )
        self.dropout = nn.Dropout(dropout)
        self.device = device

    def mask(self, x: torch.Tensor) -> torch.Tensor:
        mask = (x != self.pad_idx).unsqueeze(1).unsqueeze(2)
        return mask.to(self.device)

    def forward(self, x: torch.Tensor):
        '''
        Forward pass through architecture.

        Args:
            x: Input tensor.
        '''
        mask = self.mask(x)

        x = self.encoder(x, mask)
        x = self.dropout(x)

        x = x.max(dim=1)[0]

        out = self.linear(x)

        return out